In [ ]:
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import nltk
nltk.download('punkt')

import torchtext.vocab
import tqdm

from nltk import tokenize

from torchtext.legacy.data import Field, LabelField, TabularDataset, BucketIterator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Carregar os arquivos do GDrive

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Reprodutibilidade

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Pré-processar texto

In [ ]:
def tokenizer(text):
    return tokenize.word_tokenize(text, language='portuguese')

In [ ]:
path = "drive/MyDrive/Colab Notebooks/nn2pln/corpus/"

def process_splits():
    text = Field(tokenize=tokenizer)
    label = LabelField(dtype=torch.float)
    
    train, valid, test = TabularDataset.splits(path=path, train='balanced_train_apps.csv',
                                               validation='balanced_dev_apps.csv',
                                               test='balanced_test_apps.csv', format='csv',
                                               fields=[('text', text), ('helpfulness', label)], 
                                               skip_header=True)
   
    print(vars(test[0]))
    text.build_vocab(train)
    label.build_vocab(train)

    # string -> inteiro
    print(list(text.vocab.stoi.items())[:10])

    # inteiro -> string
    print(text.vocab.itos[:10])

    return train, valid, test, text

### Criar os iteradores

In [ ]:
train, val, test, text = process_splits()

BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# criar minibatch
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train, val, test),
                                                                      sort_key=lambda x: len(x.text), 
                                                                      batch_size=BATCH_SIZE,
                                                                      device=device)

{'text': ['App', 'ótimo', 'recomendo', '.', 'Se', 'tivesse', 'chamada', 'de', 'vedio', 'privado', 'séria', 'melhor', 'ainda', '.'], 'helpfulness': '0.0'}
[('<unk>', 0), ('<pad>', 1), (',', 2), ('.', 3), ('e', 4), ('o', 5), ('de', 6), ('que', 7), ('a', 8), ('!', 9)]
['<unk>', '<pad>', ',', '.', 'e', 'o', 'de', 'que', 'a', '!']


### Modelo

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        
        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        # embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        # embedded = [batch size, sent len, emb dim]
        
        output = F.relu(self.fc1(embedded.mean(dim=1)))
        # output = [batch size, hid dim]
        
        out = self.fc2(output)
        # out = [batch size, hid out]
        
        return out

### Parâmetros do modelo

In [ ]:
INPUT_DIM = len(text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = MLP(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

### Quantidade de parâmetros treináveis

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'O modelo tem {count_parameters(model):,} parâmetros treináveis')

O modelo tem 6,114,513 parâmetros treináveis


### Treinamento

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    
    model.train()
    
    for batch in tqdm.tqdm(iterator, desc='training...'):
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.helpfulness)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

### Avaliação

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    total_predictions = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in tqdm.tqdm(iterator, desc='evaluating...'):

            predictions = model(batch.text).squeeze(1)
            total_predictions.extend(p.item() for p in predictions)
            
            loss = criterion(predictions, batch.helpfulness)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator), total_predictions

### Execução

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss = train(model, train_iterator, optimizer, criterion)
    valid_loss, _ = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'mlp.pt')
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

evaluating...: 100%|██████████| 340/340 [00:00<00:00, 588.84it/s]


Epoch: 01
	Train Loss: 0.670
	 Val. Loss: 0.707


evaluating...: 100%|██████████| 340/340 [00:00<00:00, 561.16it/s]


Epoch: 02
	Train Loss: 0.651
	 Val. Loss: 0.731


evaluating...: 100%|██████████| 340/340 [00:00<00:00, 564.49it/s]


Epoch: 03
	Train Loss: 0.635
	 Val. Loss: 0.767


evaluating...: 100%|██████████| 340/340 [00:00<00:00, 576.71it/s]


Epoch: 04
	Train Loss: 0.623
	 Val. Loss: 0.812


evaluating...: 100%|██████████| 340/340 [00:00<00:00, 576.12it/s]

Epoch: 05
	Train Loss: 0.612
	 Val. Loss: 0.862


### Teste

In [ ]:
from sklearn.metrics import classification_report

model.load_state_dict(torch.load('mlp.pt'))

test_loss, predictions = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f}')

preds = torch.FloatTensor(predictions)
rounded_preds = torch.round(torch.sigmoid(preds))
  
labels = [float(t.helpfulness) for t in test]
print(classification_report(labels, rounded_preds.detach().numpy()))

evaluating...: 100%|██████████| 681/681 [00:01<00:00, 537.60it/s]

Test Loss: 0.706
              precision    recall  f1-score   support

         0.0       0.72      0.00      0.01     10882
         1.0       0.50      1.00      0.67     10882

    accuracy                           0.50     21764
   macro avg       0.61      0.50      0.34     21764
weighted avg       0.61      0.50      0.34     21764



### Predição

In [ ]:
def predict(sentence):
    model.eval()
    tokenized = tokenizer(sentence)
    
    # token -> int
    indexed = [text.vocab.stoi[t] for t in tokenized]

    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)

    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

print(predict('O celular é muito bom, a câmera é muito boa.'))
print(predict('A câmera é péssima.'))

0.5807092189788818
0.6065695285797119
